In [1]:
# import relevant moduls
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.manifold import TSNE
import sys
sys.path.append('../scripts/')
from analysis import vectorize_hashtags, compare_vectors

In [2]:
# read and clean files
cluster_df = pd.read_json('../../data/BTW17_Suggestions/suggestions/cluster.json')
topics_df = pd.read_json('../../data/BTW17_Twitter/lda/hashtag_topics.json')

In [3]:
# filter out all noisepoints
cluster_df = cluster_df[cluster_df['cluster']!=-1].reset_index(drop=True)
cluster_df.head(3)

,t-SNE(x),t-SNE(y),suggestion,cluster,vector
0,-34.498154,29.302818,[gebrochen],0,"[-10.9891014099, 48.0791358948]"
1,-25.847075,27.667850,"[stadt, land, fluss]",0,"[-12.6391649246, 62.3373641968]"
2,-36.280140,21.939951,"[konzert, für, dich]",0,"[30.8803348541, 18.8737182617]"


In [4]:
topics_df.head(3)

,hashtag,topic,topic_words,scores
0,btw17,60,"[erdogan, seehofer, türkei, obergrenze, mögen,...","[0.139, 0.064, 0.056, 0.030, 0.029, 0.026, 0.0..."
1,traudichdeutschland,60,"[erdogan, seehofer, türkei, obergrenze, mögen,...","[0.139, 0.064, 0.056, 0.030, 0.029, 0.026, 0.0..."
2,gauland,60,"[erdogan, seehofer, türkei, obergrenze, mögen,...","[0.139, 0.064, 0.056, 0.030, 0.029, 0.026, 0.0..."


In [5]:
for i in range(len(topics_df)):
    sum_score = 0
    for score in topics_df['scores'][i]:
        sum_score = sum_score + float(score)
    topics_df['topic_words'][i].append(topics_df['hashtag'][i])
    topics_df['scores'][i].append(sum_score)

In [7]:
hashtag_vectors = vectorize_hashtags(topics_df)

In [9]:
# tsne transformation to 2d
tsne = TSNE(n_components=2, random_state=1410)
X_tsne = tsne.fit_transform(hashtag_vectors)
topics_df['vector'] = [x for x in X_tsne]

In [10]:
similarity_df = compare_vectors(topics_df['vector'].to_numpy(), cluster_df)
similarity_df.head(3)

  0%|          | 0/17761 [00:00<?, ?it/s]

,suggestion,cluster,similarity_scores
0,[gebrochen],0,"[-0.969, -0.969, 0.136, 0.144, 0.191, 0.191, 0..."
1,"[stadt, land, fluss]",0,"[-0.974, -0.974, 0.16, 0.169, 0.215, 0.215, 0...."
2,"[konzert, für, dich]",0,"[-0.544, -0.544, 0.982, 0.984, 0.991, 0.991, 0..."


In [11]:
# save df to json
similarity_df.to_json('../../data/BTW17_Suggestions/suggestions/vector_similarity.json')
topics_df.to_json('../../data/BTW17_Twitter/hashtags/hashtag_vectors.json')